In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker
import seaborn as sns
%load_ext autoreload
%autoreload 2

from settings import *

In [2]:
df = pd.read_csv(cache_dir+'pairs_v2.tsv', sep='\t')

In [3]:
num = df[(df['related']==True) & (df['vec_sim']==0)].shape[0]

In [4]:
df = df[~((df['related']==True) & (df['vec_sim']+df['jac_sim']+df['top_sim']<0.9))]

In [5]:
df[df['related']==True].shape

(3154, 7)

In [6]:
#df = df[~df.isin(df[df['related']==False].sample(num))].dropna()

In [7]:
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
X = df[['vec_sim', 'jac_sim', 'len_sim', 'top_sim']].values
y = df[['related']].values.ravel()


fold = 10
kf = KFold(n_splits=fold, random_state=42, shuffle=True)
score = 0.0
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    #classifier = SVC(1000)
    classifier = RandomForestClassifier(n_estimators=80, max_depth=80)
    classifier.fit(X_train, y_train)
    score += classifier.score(X_test, y_test)
print(score/fold)

0.901998670086


In [8]:
classifier.feature_importances_

array([ 0.2348507 ,  0.50250872,  0.09943729,  0.16320329])